<a href="https://colab.research.google.com/github/Ana-Ibarra/OpenClassroomProject/blob/Pretrained/P6_class_images_dogs_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import shutil
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf

import keras
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.layers.normalization import BatchNormalization
from keras.callbacks import History 
history = History()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
fpath = "/content/drive/My Drive/OC_P6_images/images_squared/"
random_seed = 42
squared_size = 150
batch_size = 32
categories = os.listdir(fpath)

In [ ]:
train_ds = keras.preprocessing.image_dataset_from_directory(
    fpath,
    labels="inferred",
    label_mode='int',
    class_names=None,
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(squared_size, squared_size),
    shuffle=True,
    seed=1,
    validation_split=0.2,
    subset="training",
    interpolation="bilinear",
    follow_links=False,
)

Found 20581 files belonging to 120 classes.
Using 16465 files for training.


In [ ]:
len(train_ds)

515

In [ ]:
val_ds = keras.preprocessing.image_dataset_from_directory(
    fpath,
    labels="inferred",
    label_mode='int',
    class_names=None,
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(squared_size, squared_size),
    shuffle=True,
    seed=1,
    validation_split=0.2,
    subset="validation",
    interpolation="bilinear",
    follow_links=False)

Found 20581 files belonging to 120 classes.
Using 4116 files for validation.


Configure the dataset for performance

In [ ]:
# AUTOTUNE = tf.data.experimental.AUTOTUNE

# train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# x, x_test, y, y_test = train_test_split(train_ds, 
#                                   test_size = 0.25, train_size=0.75, 
#                                   random_state = random_seed)

In [ ]:
from tensorflow.keras import layers
data_augmentation = keras.Sequential(
        [layers.experimental.preprocessing.RandomFlip("horizontal"),
         layers.experimental.preprocessing.RandomRotation(0.1)])

In [ ]:
# import numpy as np

# for images, labels in train_ds.take(1):
#     plt.figure(figsize=(10, 10))
#     first_image = images[0]
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         augmented_image = data_augmentation(
#             tf.expand_dims(first_image, 0), training=True
#         )
#         plt.imshow(augmented_image[0].numpy().astype("int32"))
#         plt.title(int(labels[i]))
#         plt.axis("off")

In [ ]:
# tf.keras.application.Xception.preprocessinginput

In [ ]:
base_model = keras.applications.Xception(
             weights="imagenet",  # Load weights pre-trained on ImageNet.
             input_shape=(squared_size, squared_size, 3),
             include_top=False)

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(squared_size, squared_size, 3))

# Preprocess for input images as xception base model
x = keras.applications.xception.preprocess_input(inputs)

# Apply random data augmentation
x = data_augmentation(x)  

# Normalisation
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# Scale inputs to [-1, +1]
x = norm_layer(x)
norm_layer.set_weights([mean, var])

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation= "relu")(x)

# Regularize with dropout
x = keras.layers.Dropout(0.2)(x)  
outputs = keras.layers.Dense(len(categories),activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.summary()

83689472/83683744 [==============================] - 1s 0us/step
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
tf_op_layer_RealDiv (TensorF [(None, 150, 150, 3)]     0         
_________________________________________________________________
tf_op_layer_Sub (TensorFlowO [(None, 150, 150, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 150, 150, 3)       0         
_________________________________________________________________
normalization (Normalization (None, 150, 150, 3)       7         
_________________________________________________________________
xception (Functional)        (None, 5, 5, 2048)        20861480  
_______________________________________________________

In [ ]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy", 
              metrics=["accuracy"])

In [ ]:
epochs=40

callback_stop = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
checkpoint_filepath = '/tmp/checkpointPT'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
%%time
model.fit(train_ds, validation_data = val_ds, epochs=epochs, callbacks=[callback_stop, model_checkpoint_callback])

Epoch 1/40
210/515 [===========>..................] - ETA: 1:13:12 - loss: 4.7823 - accuracy: 0.0104

In [ ]:
# Unfreeze the base_model
base_model.trainable = True
model.summary()

model.compile(optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

NameError: ignored

In [ ]:
%%time
history2 = model.fit(train_ds, validation_data = val_ds, epochs=10)

SyntaxError: ignored

In [ ]:
model = load_model("/content/drive/My Drive/OC_P6_images/classe120_dogs_alex_modifDo1.h5")


NameError: ignored

In [ ]:
acc = history.history2['accuracy']
val_acc = history.history2['val_accuracy']

loss = history.history2['loss']
val_loss = history.history2['val_loss']
epochs=len(acc)
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()